# Projeto Completo de Conversor Buck com Malha Fechada

Este notebook apresenta um projeto completo de um conversor Buck DC-DC, incluindo:

1. **Especificações e Análise Teórica**
   - Cálculo dos componentes de potência
   - Análise de ripple de corrente e tensão
   - Dimensionamento do indutor e capacitor

2. **Projeto do Controlador (Malha Fechada)**
   - Modelo de pequenos sinais
   - Projeto do compensador PI
   - Análise de estabilidade

3. **Análise de Perdas**
   - Perdas de condução no MOSFET
   - Perdas de chaveamento
   - Perdas no diodo
   - Perdas no indutor

4. **Análise Térmica**
   - Modelo térmico do MOSFET
   - Dimensionamento do dissipador

5. **Simulação e Comparação**
   - Simulação com Pulsim
   - Comparação teoria vs simulação

In [1]:
from pathlib import Path
import sys

_root = Path.cwd()
_candidate_paths = []
for _ in range(6):
    for _rel in (('build-test', 'python'), ('build', 'python')):
        candidate = _root / _rel[0] / _rel[1]
        if candidate.is_dir():
            _candidate_paths.append(candidate)
    _root = _root.parent

_unique_candidates = []
_seen = set()
for candidate in _candidate_paths:
    cstr = str(candidate)
    if cstr in _seen:
        continue
    _seen.add(cstr)
    _unique_candidates.append(candidate)

_bootstrap_error = None
for candidate in _unique_candidates:
    cstr = str(candidate)
    if cstr not in sys.path:
        sys.path.insert(0, cstr)
    try:
        import pulsim  # noqa: F401
        break
    except Exception as exc:
        _bootstrap_error = exc
        if cstr in sys.path:
            sys.path.remove(cstr)
else:
    if _bootstrap_error is not None:
        raise _bootstrap_error

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pulsim

if not hasattr(pulsim, "LinearSolverKind"):
    raise RuntimeError("Kernel antigo detectado: rebuild do módulo _pulsim é necessário.")

# Configuração de plots
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print(sys.executable)
print(sys.version)
print(f"Pulsim version: {pulsim.__version__}")

/Library/Frameworks/Python.framework/Versions/3.13/bin/python3
3.13.5 (v3.13.5:6cb20a219a8, Jun 11 2025, 12:23:45) [Clang 16.0.0 (clang-1600.0.26.6)]
Pulsim version: 0.2.0


## 1. Especificações do Projeto

### 1.1 Requisitos do Sistema

In [2]:
# ============================================================
# ESPECIFICAÇÕES DO CONVERSOR BUCK
# ============================================================

# Tensões
Vin_nom = 48.0      # Tensão de entrada nominal [V]
Vin_min = 42.0      # Tensão de entrada mínima [V]
Vin_max = 54.0      # Tensão de entrada máxima [V]
Vout = 12.0         # Tensão de saída [V]

# Correntes
Iout_nom = 5.0      # Corrente de saída nominal [A]
Iout_max = 6.0      # Corrente de saída máxima [A]
Iout_min = 0.5      # Corrente de saída mínima [A]

# Ripple
delta_IL_percent = 0.30   # Ripple de corrente no indutor (30% de IL)
delta_Vout_max = 0.050    # Ripple máximo de tensão de saída [V] (50mV)

# Frequência de chaveamento
fsw = 100e3         # Frequência de chaveamento [Hz]
Tsw = 1/fsw         # Período de chaveamento [s]

# Eficiência alvo
eta_target = 0.95   # Eficiência alvo (95%)

# Temperatura ambiente
Ta = 40.0           # Temperatura ambiente [°C]
Tj_max = 125.0      # Temperatura de junção máxima [°C]

print("="*60)
print("ESPECIFICAÇÕES DO CONVERSOR BUCK")
print("="*60)
print(f"Tensão de entrada: {Vin_min}V - {Vin_max}V (nominal: {Vin_nom}V)")
print(f"Tensão de saída: {Vout}V")
print(f"Corrente de saída: {Iout_min}A - {Iout_max}A (nominal: {Iout_nom}A)")
print(f"Potência nominal: {Vout * Iout_nom:.1f}W")
print(f"Potência máxima: {Vout * Iout_max:.1f}W")
print(f"Frequência de chaveamento: {fsw/1e3:.0f}kHz")
print(f"Ripple de tensão máximo: {delta_Vout_max*1e3:.0f}mV")

ESPECIFICAÇÕES DO CONVERSOR BUCK
Tensão de entrada: 42.0V - 54.0V (nominal: 48.0V)
Tensão de saída: 12.0V
Corrente de saída: 0.5A - 6.0A (nominal: 5.0A)
Potência nominal: 60.0W
Potência máxima: 72.0W
Frequência de chaveamento: 100kHz
Ripple de tensão máximo: 50mV


### 1.2 Cálculos Teóricos - Duty Cycle e Componentes

In [3]:
# ============================================================
# ANÁLISE TEÓRICA DO CONVERSOR BUCK
# ============================================================

# Duty cycle (razão cíclica)
# Buck ideal: Vout = D * Vin
D_nom = Vout / Vin_nom
D_min = Vout / Vin_max  # Duty mínimo quando Vin é máximo
D_max = Vout / Vin_min  # Duty máximo quando Vin é mínimo

print("="*60)
print("ANÁLISE DE DUTY CYCLE")
print("="*60)
print(f"D nominal (Vin={Vin_nom}V): {D_nom:.3f} ({D_nom*100:.1f}%)")
print(f"D mínimo (Vin={Vin_max}V): {D_min:.3f} ({D_min*100:.1f}%)")
print(f"D máximo (Vin={Vin_min}V): {D_max:.3f} ({D_max*100:.1f}%)")

# Corrente média no indutor = corrente de saída (em CCM)
IL_avg = Iout_nom

# Ripple de corrente no indutor
delta_IL = delta_IL_percent * IL_avg

# Correntes de pico e vale
IL_max = IL_avg + delta_IL / 2
IL_min = IL_avg - delta_IL / 2

print(f"\nCorrente média no indutor: {IL_avg:.2f}A")
print(f"Ripple de corrente: {delta_IL:.2f}A ({delta_IL_percent*100:.0f}%)")
print(f"Corrente de pico: {IL_max:.2f}A")
print(f"Corrente de vale: {IL_min:.2f}A")

ANÁLISE DE DUTY CYCLE
D nominal (Vin=48.0V): 0.250 (25.0%)
D mínimo (Vin=54.0V): 0.222 (22.2%)
D máximo (Vin=42.0V): 0.286 (28.6%)

Corrente média no indutor: 5.00A
Ripple de corrente: 1.50A (30%)
Corrente de pico: 5.75A
Corrente de vale: 4.25A


In [4]:
# ============================================================
# DIMENSIONAMENTO DO INDUTOR
# ============================================================

# Fórmula do ripple de corrente:
# delta_IL = (Vin - Vout) * D * Tsw / L = Vout * (1-D) * Tsw / L
#
# Resolvendo para L:
# L = Vout * (1-D) * Tsw / delta_IL

# Usando Vin_max (pior caso para ripple de corrente, menor D)
L_min = Vout * (1 - D_min) / (delta_IL * fsw)

# Valor comercial (arredondando para cima)
L = np.ceil(L_min * 1e6) * 1e-6  # Arredonda para próximo uH

# Recalculando ripple real com valor comercial
delta_IL_real = Vout * (1 - D_min) / (L * fsw)

print("="*60)
print("DIMENSIONAMENTO DO INDUTOR")
print("="*60)
print(f"Indutância mínima calculada: {L_min*1e6:.2f}µH")
print(f"Indutância selecionada: {L*1e6:.0f}µH")
print(f"Ripple de corrente real: {delta_IL_real:.2f}A ({delta_IL_real/IL_avg*100:.1f}%)")

# Verificação de CCM (Modo de Condução Contínua)
IL_min_ccm = delta_IL_real / 2
print(f"\nCorrente mínima para CCM: {IL_min_ccm:.2f}A")
print(f"Modo de operação em Iout_min={Iout_min}A: {'CCM' if Iout_min > IL_min_ccm else 'DCM'}")

DIMENSIONAMENTO DO INDUTOR
Indutância mínima calculada: 62.22µH
Indutância selecionada: 63µH
Ripple de corrente real: 1.48A (29.6%)

Corrente mínima para CCM: 0.74A
Modo de operação em Iout_min=0.5A: DCM


In [5]:
# ============================================================
# DIMENSIONAMENTO DO CAPACITOR DE SAÍDA
# ============================================================

# O ripple de tensão de saída vem de duas fontes:
# 1. Carga/descarga do capacitor (delta_Vc)
# 2. Queda na ESR do capacitor (delta_V_ESR)
#
# delta_Vout = delta_Vc + delta_V_ESR
#
# Para capacitor ideal (ESR = 0):
# delta_Vc = delta_IL * Tsw / (8 * C)

# Assumindo que metade do ripple vem da capacitância e metade da ESR
delta_Vc_max = delta_Vout_max * 0.7  # 70% do ripple do capacitor
delta_V_ESR_max = delta_Vout_max * 0.3  # 30% do ripple da ESR

# Capacitância mínima
C_min = delta_IL_real / (8 * fsw * delta_Vc_max)

# ESR máxima
ESR_max = delta_V_ESR_max / delta_IL_real

# Valor comercial (capacitor eletrolítico de baixa ESR ou cerâmico)
C = np.ceil(C_min * 1e6 / 10) * 10e-6  # Arredonda para próximo 10uF
ESR = 0.020  # 20mΩ (típico para eletrolítico de baixa ESR)

print("="*60)
print("DIMENSIONAMENTO DO CAPACITOR DE SAÍDA")
print("="*60)
print(f"Capacitância mínima calculada: {C_min*1e6:.1f}µF")
print(f"ESR máxima: {ESR_max*1e3:.1f}mΩ")
print(f"Capacitância selecionada: {C*1e6:.0f}µF")
print(f"ESR assumida: {ESR*1e3:.1f}mΩ")

# Ripple real
delta_Vc_real = delta_IL_real / (8 * fsw * C)
delta_V_ESR_real = delta_IL_real * ESR
delta_Vout_real = delta_Vc_real + delta_V_ESR_real

print(f"\nRipple de tensão real:")
print(f"  - Contribuição da capacitância: {delta_Vc_real*1e3:.1f}mV")
print(f"  - Contribuição da ESR: {delta_V_ESR_real*1e3:.1f}mV")
print(f"  - Total: {delta_Vout_real*1e3:.1f}mV (máx: {delta_Vout_max*1e3:.0f}mV)")

DIMENSIONAMENTO DO CAPACITOR DE SAÍDA
Capacitância mínima calculada: 52.9µF
ESR máxima: 10.1mΩ
Capacitância selecionada: 60µF
ESR assumida: 20.0mΩ

Ripple de tensão real:
  - Contribuição da capacitância: 30.9mV
  - Contribuição da ESR: 29.6mV
  - Total: 60.5mV (máx: 50mV)


In [6]:
# ============================================================
# RESUMO DOS COMPONENTES
# ============================================================

print("="*60)
print("RESUMO DOS COMPONENTES DE POTÊNCIA")
print("="*60)
print(f"\nINDUTOR:")
print(f"  L = {L*1e6:.0f}µH")
print(f"  Corrente RMS: {np.sqrt(IL_avg**2 + (delta_IL_real**2)/12):.2f}A")
print(f"  Corrente de pico: {IL_avg + delta_IL_real/2:.2f}A")

print(f"\nCAPACITOR:")
print(f"  C = {C*1e6:.0f}µF")
print(f"  Tensão nominal: ≥{Vout*1.5:.0f}V")
print(f"  ESR máxima: {ESR_max*1e3:.1f}mΩ")
print(f"  Corrente RMS: {delta_IL_real/np.sqrt(12):.2f}A")

print(f"\nMOSFET (High-Side):")
print(f"  Vds_max ≥ {Vin_max*1.3:.0f}V (com margem)")
print(f"  Id_max ≥ {(IL_avg + delta_IL_real/2)*1.5:.1f}A (com margem)")
print(f"  Corrente RMS: {IL_avg * np.sqrt(D_nom):.2f}A")

print(f"\nDIODO (Low-Side):")
print(f"  Vrrm ≥ {Vin_max*1.3:.0f}V")
print(f"  If_avg: {IL_avg * (1-D_nom):.2f}A")

RESUMO DOS COMPONENTES DE POTÊNCIA

INDUTOR:
  L = 63µH
  Corrente RMS: 5.02A
  Corrente de pico: 5.74A

CAPACITOR:
  C = 60µF
  Tensão nominal: ≥18V
  ESR máxima: 10.1mΩ
  Corrente RMS: 0.43A

MOSFET (High-Side):
  Vds_max ≥ 70V (com margem)
  Id_max ≥ 8.6A (com margem)
  Corrente RMS: 2.50A

DIODO (Low-Side):
  Vrrm ≥ 70V
  If_avg: 3.75A


## 2. Projeto do Controlador (Malha Fechada)

### 2.1 Modelo de Pequenos Sinais

In [7]:
# ============================================================
# MODELO DE PEQUENOS SINAIS DO BUCK
# ============================================================

# Resistência de carga
R_load = Vout / Iout_nom

# Função de transferência controle-para-saída (Gvd):
# Gvd(s) = Vout/d = Vin * (1 + s*ESR*C) / (1 + s/Q/w0 + s²/w0²)
#
# Onde:
# w0 = 1/sqrt(L*C)  - frequência de ressonância LC
# Q = R * sqrt(C/L) - fator de qualidade
# wz = 1/(ESR*C)    - zero da ESR

w0 = 1 / np.sqrt(L * C)
f0 = w0 / (2 * np.pi)
Q = R_load * np.sqrt(C / L)
wz = 1 / (ESR * C)
fz = wz / (2 * np.pi)

print("="*60)
print("MODELO DE PEQUENOS SINAIS")
print("="*60)
print(f"Resistência de carga: R = {R_load:.2f}Ω")
print(f"Frequência de ressonância LC: f0 = {f0:.0f}Hz")
print(f"Fator de qualidade: Q = {Q:.2f}")
print(f"Zero da ESR: fz = {fz/1e3:.1f}kHz")
print(f"Frequência de chaveamento: fsw = {fsw/1e3:.0f}kHz")

# Criar função de transferência Gvd(s)
# Numerador: Vin * (1 + s*ESR*C)
# Denominador: 1 + s/(Q*w0) + s²/w0²

Gvd_num = [Vin_nom * ESR * C, Vin_nom]
Gvd_den = [1/w0**2, 1/(Q*w0), 1]
Gvd = signal.TransferFunction(Gvd_num, Gvd_den)

print(f"\nFunção de transferência Gvd(s):")
print(f"  Gvd(s) = {Vin_nom:.0f} * (1 + s/{wz:.0f}) / (1 + s/{Q*w0:.0f} + s²/{w0**2:.0f})")

MODELO DE PEQUENOS SINAIS
Resistência de carga: R = 2.40Ω
Frequência de ressonância LC: f0 = 2589Hz
Fator de qualidade: Q = 2.34
Zero da ESR: fz = 132.6kHz
Frequência de chaveamento: fsw = 100kHz

Função de transferência Gvd(s):
  Gvd(s) = 48 * (1 + s/833333) / (1 + s/38095 + s²/264550265)


In [8]:
# ============================================================
# DIAGRAMA DE BODE DA PLANTA
# ============================================================

# Frequências para análise
f = np.logspace(1, 5, 1000)  # 10Hz a 100kHz
w = 2 * np.pi * f

# Resposta em frequência
w_out, H = signal.freqresp(Gvd, w)
mag_db = 20 * np.log10(np.abs(H))
phase_deg = np.angle(H, deg=True)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.semilogx(f, mag_db, 'b-', linewidth=2, label='Gvd(s)')
ax1.axvline(f0, color='r', linestyle='--', alpha=0.7, label=f'f0 = {f0:.0f}Hz')
ax1.axvline(fz, color='g', linestyle='--', alpha=0.7, label=f'fz = {fz/1e3:.1f}kHz')
ax1.axvline(fsw, color='orange', linestyle='--', alpha=0.7, label=f'fsw = {fsw/1e3:.0f}kHz')
ax1.set_ylabel('Magnitude [dB]')
ax1.set_title('Diagrama de Bode - Planta Gvd(s)')
ax1.legend(loc='upper right')
ax1.set_ylim([-60, 60])

ax2.semilogx(f, phase_deg, 'b-', linewidth=2)
ax2.axvline(f0, color='r', linestyle='--', alpha=0.7)
ax2.axvline(fz, color='g', linestyle='--', alpha=0.7)
ax2.axvline(fsw, color='orange', linestyle='--', alpha=0.7)
ax2.set_xlabel('Frequência [Hz]')
ax2.set_ylabel('Fase [°]')
ax2.set_ylim([-180, 0])

plt.tight_layout()
plt.show()

print(f"Ganho DC da planta: {20*np.log10(Vin_nom):.1f}dB ({Vin_nom:.0f}V/V)")

Ganho DC da planta: 33.6dB (48V/V)


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_88225/716912040.py:35: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 2.2 Projeto do Compensador PI

In [9]:
# ============================================================
# PROJETO DO COMPENSADOR PI
# ============================================================

# Especificações de malha fechada
fc = fsw / 10  # Frequência de cruzamento: fsw/10 (regra prática)
PM_target = 60  # Margem de fase desejada [graus]

print("="*60)
print("PROJETO DO COMPENSADOR PI")
print("="*60)
print(f"Frequência de cruzamento alvo: fc = {fc/1e3:.1f}kHz")
print(f"Margem de fase desejada: PM = {PM_target}°")

# Compensador PI: Gc(s) = Kp * (1 + 1/(Ti*s)) = Kp * (1 + wz/s)
#                      = Kp * (s + wz) / s
#
# O compensador PI adiciona:
# - Um polo na origem (integrador) -> +90° de atraso em baixa freq
# - Um zero em wz = 1/Ti -> recupera fase

# Passo 1: Calcular ganho da planta em fc
wc = 2 * np.pi * fc
_, Gvd_fc = signal.freqresp(Gvd, [wc])
Gvd_mag_fc = np.abs(Gvd_fc[0])
Gvd_phase_fc = np.angle(Gvd_fc[0], deg=True)

print(f"\nPlanta em fc = {fc/1e3:.1f}kHz:")
print(f"  |Gvd(fc)| = {20*np.log10(Gvd_mag_fc):.1f}dB")
print(f"  ∠Gvd(fc) = {Gvd_phase_fc:.1f}°")

# Passo 2: Posicionar o zero do PI
# Zero em f0/2 para compensar o atraso do LC sem adicionar muito ganho
fz_pi = f0 / 2
wz_pi = 2 * np.pi * fz_pi
Ti = 1 / wz_pi  # Constante de tempo integrativa

# Passo 3: Calcular Kp para ganho unitário em fc
# |Gc(fc)| * |Gvd(fc)| = 1
# |Gc(fc)| = Kp * |1 + j*fc/fz_pi| / |j*fc/fz_pi|
#          ≈ Kp * fc/fz_pi para fc >> fz_pi

Gc_mag_at_fc = np.sqrt(1 + (fc/fz_pi)**2) / (fc/fz_pi)  # Magnitude normalizada do PI
Kp = 1 / (Gvd_mag_fc * Gc_mag_at_fc)

# Ganho integral
Ki = Kp / Ti

print(f"\nParâmetros do compensador PI:")
print(f"  Zero do PI: fz = {fz_pi:.0f}Hz")
print(f"  Kp = {Kp:.4f}")
print(f"  Ki = {Ki:.4f}")
print(f"  Ti = {Ti*1e3:.3f}ms")

# Criar função de transferência do compensador PI
# Gc(s) = Kp + Ki/s = (Kp*s + Ki)/s
Gc_num = [Kp, Ki]
Gc_den = [1, 0]
Gc = signal.TransferFunction(Gc_num, Gc_den)

PROJETO DO COMPENSADOR PI
Frequência de cruzamento alvo: fc = 10.0kHz
Margem de fase desejada: PM = 60°

Planta em fc = 10.0kHz:
  |Gvd(fc)| = 10.7dB
  ∠Gvd(fc) = -168.9°

Parâmetros do compensador PI:
  Zero do PI: fz = 1294Hz
  Kp = 0.2889
  Ki = 2349.0823
  Ti = 0.123ms


In [10]:
# ============================================================
# ANÁLISE DE MALHA ABERTA
# ============================================================

# Ganho de malha: T(s) = Gc(s) * Gvd(s)
T_num = np.convolve(Gc_num, Gvd_num)
T_den = np.convolve(Gc_den, Gvd_den)
T = signal.TransferFunction(T_num, T_den)

# Resposta em frequência
_, Gc_H = signal.freqresp(Gc, w)
_, T_H = signal.freqresp(T, w)

Gc_mag_db = 20 * np.log10(np.abs(Gc_H))
Gc_phase_deg = np.angle(Gc_H, deg=True)
T_mag_db = 20 * np.log10(np.abs(T_H))
T_phase_deg = np.angle(T_H, deg=True)

# Encontrar frequência de cruzamento real e margem de fase
idx_crossover = np.argmin(np.abs(T_mag_db))
fc_real = f[idx_crossover]
PM_real = 180 + T_phase_deg[idx_crossover]

# Encontrar margem de ganho (onde fase = -180°)
idx_180 = np.argmin(np.abs(T_phase_deg + 180))
GM_real = -T_mag_db[idx_180]
f_180 = f[idx_180]

print("="*60)
print("ANÁLISE DE ESTABILIDADE")
print("="*60)
print(f"Frequência de cruzamento real: fc = {fc_real/1e3:.2f}kHz")
print(f"Margem de fase: PM = {PM_real:.1f}° (alvo: {PM_target}°)")
print(f"Margem de ganho: GM = {GM_real:.1f}dB @ {f_180/1e3:.1f}kHz")

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.semilogx(f, mag_db, 'b--', linewidth=1.5, alpha=0.7, label='Planta Gvd(s)')
ax1.semilogx(f, Gc_mag_db, 'g--', linewidth=1.5, alpha=0.7, label='Compensador Gc(s)')
ax1.semilogx(f, T_mag_db, 'r-', linewidth=2, label='Malha T(s)=Gc*Gvd')
ax1.axhline(0, color='k', linestyle='-', alpha=0.3)
ax1.axvline(fc_real, color='purple', linestyle=':', alpha=0.7)
ax1.plot(fc_real, 0, 'ro', markersize=10, label=f'fc={fc_real/1e3:.1f}kHz')
ax1.set_ylabel('Magnitude [dB]')
ax1.set_title('Diagrama de Bode - Malha Aberta')
ax1.legend(loc='upper right')
ax1.set_ylim([-60, 80])

ax2.semilogx(f, phase_deg, 'b--', linewidth=1.5, alpha=0.7)
ax2.semilogx(f, Gc_phase_deg, 'g--', linewidth=1.5, alpha=0.7)
ax2.semilogx(f, T_phase_deg, 'r-', linewidth=2)
ax2.axhline(-180, color='k', linestyle='-', alpha=0.3)
ax2.axvline(fc_real, color='purple', linestyle=':', alpha=0.7)
ax2.plot(fc_real, T_phase_deg[idx_crossover], 'ro', markersize=10)
ax2.annotate(f'PM={PM_real:.0f}°', xy=(fc_real, T_phase_deg[idx_crossover]),
             xytext=(fc_real*2, T_phase_deg[idx_crossover]+20),
             fontsize=12, arrowprops=dict(arrowstyle='->', color='red'))
ax2.set_xlabel('Frequência [Hz]')
ax2.set_ylabel('Fase [°]')
ax2.set_ylim([-270, 90])

plt.tight_layout()
plt.show()

ANÁLISE DE ESTABILIDADE
Frequência de cruzamento real: fc = 9.98kHz
Margem de fase: PM = 3.7° (alvo: 60°)
Margem de ganho: GM = -6.8dB @ 7.0kHz


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_88225/3945261362.py:64: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 3. Análise de Perdas

### 3.1 Seleção de Componentes

In [11]:
# ============================================================
# SELEÇÃO DE COMPONENTES SEMICONDUTORES
# ============================================================

# MOSFET: IRF540N (100V, 33A, 44mΩ)
mosfet_params = {
    'name': 'IRF540N',
    'Vds_max': 100,      # V
    'Id_max': 33,        # A
    'Rds_on': 0.044,     # Ω @ Vgs=10V, Tj=25°C
    'Rds_on_hot': 0.077, # Ω @ Tj=100°C (1.75x factor)
    'Qg': 71e-9,         # C (carga de gate total)
    'Qgd': 29e-9,        # C (carga gate-drain, Miller)
    'tr': 54e-9,         # s (tempo de subida)
    'tf': 39e-9,         # s (tempo de descida)
    'Coss': 360e-12,     # F (capacitância de saída)
    'Rth_jc': 1.0,       # °C/W (resistência térmica junção-case)
    'Rth_cs': 0.5,       # °C/W (resistência térmica case-sink)
}

# Diodo: MUR860 (600V, 8A, ultrafast)
diode_params = {
    'name': 'MUR860',
    'Vrrm': 600,         # V
    'If_avg': 8,         # A
    'Vf': 0.9,           # V @ If=1A
    'Vf_hot': 0.75,      # V @ Tj=100°C
    'trr': 50e-9,        # s (tempo de recuperação reversa)
    'Qrr': 100e-9,       # C (carga de recuperação)
    'Rth_jc': 2.0,       # °C/W
}

# Indutor (núcleo de ferrite)
inductor_params = {
    'L': L,
    'DCR': 0.015,        # Ω (resistência DC do enrolamento)
    'Isat': 10,          # A (corrente de saturação)
}

print("="*60)
print("COMPONENTES SELECIONADOS")
print("="*60)
print(f"\nMOSFET: {mosfet_params['name']}")
print(f"  Vds = {mosfet_params['Vds_max']}V, Id = {mosfet_params['Id_max']}A")
print(f"  Rds(on) = {mosfet_params['Rds_on']*1e3:.0f}mΩ @ 25°C")
print(f"  Rds(on) = {mosfet_params['Rds_on_hot']*1e3:.0f}mΩ @ 100°C")

print(f"\nDIODO: {diode_params['name']}")
print(f"  Vrrm = {diode_params['Vrrm']}V, If = {diode_params['If_avg']}A")
print(f"  Vf = {diode_params['Vf']}V, trr = {diode_params['trr']*1e9:.0f}ns")

COMPONENTES SELECIONADOS

MOSFET: IRF540N
  Vds = 100V, Id = 33A
  Rds(on) = 44mΩ @ 25°C
  Rds(on) = 77mΩ @ 100°C

DIODO: MUR860
  Vrrm = 600V, If = 8A
  Vf = 0.9V, trr = 50ns


In [12]:
# ============================================================
# CÁLCULO DE PERDAS
# ============================================================

# Parâmetros de operação
Vin = Vin_nom
D = D_nom
IL = IL_avg

print("="*60)
print("ANÁLISE DE PERDAS (Condição Nominal)")
print("="*60)
print(f"Vin = {Vin}V, Vout = {Vout}V, Iout = {IL}A")
print(f"D = {D:.3f}, fsw = {fsw/1e3:.0f}kHz")

# ----- PERDAS NO MOSFET -----
print("\n--- PERDAS NO MOSFET ---")

# Corrente RMS no MOSFET
I_mosfet_rms = IL * np.sqrt(D)

# Perda de condução
P_cond_mosfet = I_mosfet_rms**2 * mosfet_params['Rds_on_hot']
print(f"Corrente RMS: {I_mosfet_rms:.2f}A")
print(f"Perda de condução: {P_cond_mosfet:.3f}W")

# Perda de chaveamento (aproximação linear)
# Psw = 0.5 * Vin * IL * (tr + tf) * fsw
tr = mosfet_params['tr']
tf = mosfet_params['tf']
P_sw_mosfet = 0.5 * Vin * IL * (tr + tf) * fsw
print(f"Perda de chaveamento: {P_sw_mosfet:.3f}W")

# Perda de gate drive
Vgs = 10.0  # Tensão de gate
P_gate = mosfet_params['Qg'] * Vgs * fsw
print(f"Perda de gate drive: {P_gate:.3f}W")

# Perda por Coss
P_coss = 0.5 * mosfet_params['Coss'] * Vin**2 * fsw
print(f"Perda por Coss: {P_coss:.3f}W")

P_mosfet_total = P_cond_mosfet + P_sw_mosfet + P_gate + P_coss
print(f"\nPERDA TOTAL NO MOSFET: {P_mosfet_total:.3f}W")

# ----- PERDAS NO DIODO -----
print("\n--- PERDAS NO DIODO ---")

# Corrente média no diodo
I_diode_avg = IL * (1 - D)

# Perda de condução
P_cond_diode = I_diode_avg * diode_params['Vf_hot']
print(f"Corrente média: {I_diode_avg:.2f}A")
print(f"Perda de condução: {P_cond_diode:.3f}W")

# Perda de recuperação reversa
P_rr_diode = 0.5 * diode_params['Qrr'] * Vin * fsw
print(f"Perda de recuperação: {P_rr_diode:.3f}W")

P_diode_total = P_cond_diode + P_rr_diode
print(f"\nPERDA TOTAL NO DIODO: {P_diode_total:.3f}W")

# ----- PERDAS NO INDUTOR -----
print("\n--- PERDAS NO INDUTOR ---")

# Corrente RMS no indutor (aproximadamente igual à média para baixo ripple)
I_L_rms = np.sqrt(IL**2 + (delta_IL_real**2)/12)

# Perda de cobre (DCR)
P_copper = I_L_rms**2 * inductor_params['DCR']
print(f"Corrente RMS: {I_L_rms:.2f}A")
print(f"Perda de cobre (DCR): {P_copper:.3f}W")

# Perda no núcleo (estimativa simplificada ~5% das perdas de cobre para ferrite)
P_core = P_copper * 0.1  # Aproximação
print(f"Perda no núcleo (estimativa): {P_core:.3f}W")

P_inductor_total = P_copper + P_core
print(f"\nPERDA TOTAL NO INDUTOR: {P_inductor_total:.3f}W")

# ----- PERDAS NO CAPACITOR -----
print("\n--- PERDAS NO CAPACITOR ---")

# Corrente RMS no capacitor (ripple triangular)
I_C_rms = delta_IL_real / np.sqrt(12)
P_cap = I_C_rms**2 * ESR
print(f"Corrente RMS: {I_C_rms:.3f}A")
print(f"Perda na ESR: {P_cap:.4f}W")

ANÁLISE DE PERDAS (Condição Nominal)
Vin = 48.0V, Vout = 12.0V, Iout = 5.0A
D = 0.250, fsw = 100kHz

--- PERDAS NO MOSFET ---
Corrente RMS: 2.50A
Perda de condução: 0.481W
Perda de chaveamento: 1.116W
Perda de gate drive: 0.071W
Perda por Coss: 0.041W

PERDA TOTAL NO MOSFET: 1.710W

--- PERDAS NO DIODO ---
Corrente média: 3.75A
Perda de condução: 2.812W
Perda de recuperação: 0.240W

PERDA TOTAL NO DIODO: 3.053W

--- PERDAS NO INDUTOR ---
Corrente RMS: 5.02A
Perda de cobre (DCR): 0.378W
Perda no núcleo (estimativa): 0.038W

PERDA TOTAL NO INDUTOR: 0.416W

--- PERDAS NO CAPACITOR ---
Corrente RMS: 0.428A
Perda na ESR: 0.0037W


In [13]:
# ============================================================
# RESUMO DE PERDAS E EFICIÊNCIA
# ============================================================

P_total = P_mosfet_total + P_diode_total + P_inductor_total + P_cap
P_out = Vout * Iout_nom
P_in = P_out + P_total
eta = P_out / P_in

print("="*60)
print("RESUMO DE PERDAS E EFICIÊNCIA")
print("="*60)

# Tabela de perdas
losses = [
    ('MOSFET - Condução', P_cond_mosfet),
    ('MOSFET - Chaveamento', P_sw_mosfet),
    ('MOSFET - Gate', P_gate),
    ('MOSFET - Coss', P_coss),
    ('Diodo - Condução', P_cond_diode),
    ('Diodo - Recuperação', P_rr_diode),
    ('Indutor - Cobre', P_copper),
    ('Indutor - Núcleo', P_core),
    ('Capacitor - ESR', P_cap),
]

print(f"\n{'Componente':<25} {'Perda [W]':>10} {'Percentual':>10}")
print("-" * 47)
for name, loss in losses:
    print(f"{name:<25} {loss:>10.3f} {loss/P_total*100:>9.1f}%")
print("-" * 47)
print(f"{'TOTAL':<25} {P_total:>10.3f} {100:>9.1f}%")

print(f"\nPotência de saída: {P_out:.1f}W")
print(f"Potência de entrada: {P_in:.1f}W")
print(f"Perdas totais: {P_total:.2f}W")
print(f"\nEFICIÊNCIA: {eta*100:.2f}% (alvo: {eta_target*100:.0f}%)")

# Gráfico de pizza das perdas
fig, ax = plt.subplots(figsize=(10, 8))
labels = [name for name, _ in losses]
sizes = [loss for _, loss in losses]
colors = plt.cm.Set3(np.linspace(0, 1, len(losses)))
explode = [0.02] * len(losses)

wedges, texts, autotexts = ax.pie(sizes, explode=explode, labels=labels, colors=colors,
                                   autopct='%1.1f%%', shadow=True, startangle=90)
ax.set_title(f'Distribuição de Perdas (Total: {P_total:.2f}W)\nEficiência: {eta*100:.1f}%')
plt.tight_layout()
plt.show()

RESUMO DE PERDAS E EFICIÊNCIA

Componente                 Perda [W] Percentual
-----------------------------------------------
MOSFET - Condução              0.481       9.3%
MOSFET - Chaveamento           1.116      21.5%
MOSFET - Gate                  0.071       1.4%
MOSFET - Coss                  0.041       0.8%
Diodo - Condução               2.812      54.3%
Diodo - Recuperação            0.240       4.6%
Indutor - Cobre                0.378       7.3%
Indutor - Núcleo               0.038       0.7%
Capacitor - ESR                0.004       0.1%
-----------------------------------------------
TOTAL                          5.181     100.0%

Potência de saída: 60.0W
Potência de entrada: 65.2W
Perdas totais: 5.18W

EFICIÊNCIA: 92.05% (alvo: 95%)


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_88225/890271011.py:50: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 4. Análise Térmica

In [14]:
# ============================================================
# ANÁLISE TÉRMICA
# ============================================================

print("="*60)
print("ANÁLISE TÉRMICA")
print("="*60)

# Resistências térmicas
Rth_jc_mosfet = mosfet_params['Rth_jc']  # Junção-Case
Rth_cs_mosfet = mosfet_params['Rth_cs']  # Case-Sink
Rth_jc_diode = diode_params['Rth_jc']

# Sem dissipador (junção ao ar)
Rth_ja_mosfet = 62  # °C/W (típico para TO-220 sem dissipador)
Rth_ja_diode = 50   # °C/W

# Temperatura de junção SEM dissipador
Tj_mosfet_no_sink = Ta + P_mosfet_total * Rth_ja_mosfet
Tj_diode_no_sink = Ta + P_diode_total * Rth_ja_diode

print(f"\n--- SEM DISSIPADOR ---")
print(f"Temperatura ambiente: Ta = {Ta}°C")
print(f"\nMOSFET (P = {P_mosfet_total:.2f}W):")
print(f"  Rth(j-a) = {Rth_ja_mosfet}°C/W")
print(f"  Tj = {Tj_mosfet_no_sink:.1f}°C {'⚠️ EXCEDE LIMITE!' if Tj_mosfet_no_sink > Tj_max else '✓'}")

print(f"\nDIODO (P = {P_diode_total:.2f}W):")
print(f"  Rth(j-a) = {Rth_ja_diode}°C/W")
print(f"  Tj = {Tj_diode_no_sink:.1f}°C {'⚠️ EXCEDE LIMITE!' if Tj_diode_no_sink > Tj_max else '✓'}")

# Dimensionamento do dissipador (se necessário)
if Tj_mosfet_no_sink > Tj_max or Tj_diode_no_sink > Tj_max:
    print(f"\n--- DIMENSIONAMENTO DO DISSIPADOR ---")
    
    # Considerando MOSFET e Diodo no mesmo dissipador
    P_total_sink = P_mosfet_total + P_diode_total
    Tj_target = 100  # Temperatura de junção alvo
    
    # Para simplificar, considerar o pior caso (MOSFET)
    # Tj = Ta + P * (Rth_jc + Rth_cs + Rth_sa)
    # Rth_sa = (Tj - Ta)/P - Rth_jc - Rth_cs
    
    Rth_sa_max_mosfet = (Tj_target - Ta)/P_mosfet_total - Rth_jc_mosfet - Rth_cs_mosfet
    Rth_sa_max_diode = (Tj_target - Ta)/P_diode_total - Rth_jc_diode - 0.5
    
    # Para dissipador compartilhado:
    # Aproximação: Rth_sa_shared = (Tj - Ta) / P_total - média das Rth_jc
    Rth_sa_shared = (Tj_target - Ta) / P_total_sink - (Rth_jc_mosfet + Rth_jc_diode)/2 - 0.5
    
    print(f"Potência total no dissipador: {P_total_sink:.2f}W")
    print(f"Tj alvo: {Tj_target}°C")
    print(f"\nRth(s-a) máxima:")
    print(f"  MOSFET individual: {Rth_sa_max_mosfet:.1f}°C/W")
    print(f"  Diodo individual: {Rth_sa_max_diode:.1f}°C/W")
    print(f"  Dissipador compartilhado: ≤{Rth_sa_shared:.1f}°C/W")
    
    # Temperatura real com dissipador
    Rth_sa_selected = 5.0  # °C/W (dissipador médio)
    
    Ts = Ta + P_total_sink * Rth_sa_selected  # Temp do dissipador
    Tj_mosfet_with_sink = Ts + P_mosfet_total * (Rth_jc_mosfet + Rth_cs_mosfet)
    Tj_diode_with_sink = Ts + P_diode_total * (Rth_jc_diode + 0.5)
    
    print(f"\n--- COM DISSIPADOR (Rth_sa = {Rth_sa_selected}°C/W) ---")
    print(f"Temperatura do dissipador: Ts = {Ts:.1f}°C")
    print(f"Tj MOSFET: {Tj_mosfet_with_sink:.1f}°C {'✓' if Tj_mosfet_with_sink < Tj_max else '⚠️'}")
    print(f"Tj Diodo: {Tj_diode_with_sink:.1f}°C {'✓' if Tj_diode_with_sink < Tj_max else '⚠️'}")

ANÁLISE TÉRMICA

--- SEM DISSIPADOR ---
Temperatura ambiente: Ta = 40.0°C

MOSFET (P = 1.71W):
  Rth(j-a) = 62°C/W
  Tj = 146.0°C ⚠️ EXCEDE LIMITE!

DIODO (P = 3.05W):
  Rth(j-a) = 50°C/W
  Tj = 192.6°C ⚠️ EXCEDE LIMITE!

--- DIMENSIONAMENTO DO DISSIPADOR ---
Potência total no dissipador: 4.76W
Tj alvo: 100°C

Rth(s-a) máxima:
  MOSFET individual: 33.6°C/W
  Diodo individual: 17.2°C/W
  Dissipador compartilhado: ≤10.6°C/W

--- COM DISSIPADOR (Rth_sa = 5.0°C/W) ---
Temperatura do dissipador: Ts = 63.8°C
Tj MOSFET: 66.4°C ✓
Tj Diodo: 71.4°C ✓


## 5. Simulação com Pulsim

### 5.1 Construção do Circuito

In [15]:
# ============================================================
# SIMULAÇÃO COM PULSIM - CIRCUITO PWM REAL
# ============================================================

print("="*60)
print("CONSTRUÇÃO DO CIRCUITO BUCK COM PWM REAL")
print("="*60)

# Agora com suporte a fontes PWM, podemos simular o conversor buck
# com chaveamento real em vez do modelo DC equivalente!

# Criar circuito
ckt = pulsim.Circuit()

# Definir nós - TOPOLOGIA CORRETA DO BUCK:
#
#    PWM (sw)                          out
#      |                                |
#      +---[DCR]---[L]---+---[Rload]---+
#                        |              |
#                       [ESR]          GND
#                        |
#                       [C]
#                        |
#                       GND
#
# sw: nó de chaveamento (saída do PWM)
# mid: nó entre DCR e indutor
# out: saída filtrada (APÓS o indutor)
# cap: nó entre ESR e capacitor

n_sw = ckt.add_node("sw")       # Nó de chaveamento (PWM output)
n_mid = ckt.add_node("mid")     # Entre DCR e indutor
n_out = ckt.add_node("out")     # Saída filtrada (APÓS LC filter)
n_cap = ckt.add_node("cap")     # Nó do capacitor (após ESR)

# PWM fonte que emula o comportamento do switch high-side
# Quando ON: V_sw = Vin, Quando OFF: V_sw = 0 (diodo conduz)
pwm_params = pulsim.PWMParams()
pwm_params.v_high = Vin_nom     # Tensão quando switch ON (48V)
pwm_params.v_low = 0.0          # Tensão quando switch OFF (diodo)
pwm_params.frequency = fsw      # 100kHz
pwm_params.duty = D_nom         # Duty cycle nominal (0.25)
pwm_params.rise_time = 50e-9    # 50ns rise time
pwm_params.fall_time = 50e-9    # 50ns fall time

ckt.add_pwm_voltage_source("Vpwm", n_sw, ckt.ground(), pwm_params)

# Caminho do indutor: sw -> DCR -> L -> out
# DCR em série com o indutor
ckt.add_resistor("Rdcr", n_sw, n_mid, inductor_params['DCR'])
ckt.add_inductor("L1", n_mid, n_out, L)

# Capacitor com ESR: out -> ESR -> C -> gnd
# ESR em série com o capacitor
ckt.add_resistor("Resr", n_out, n_cap, ESR)
ckt.add_capacitor("Cout", n_cap, ckt.ground(), C)

# Carga resistiva: out -> gnd
ckt.add_resistor("Rload", n_out, ckt.ground(), R_load)

print(f"Circuito criado com {ckt.num_devices()} dispositivos:")
print(f"  - Nós: {ckt.num_nodes()} ({', '.join(ckt.node_names())})")
print(f"  - Tamanho do sistema: {ckt.system_size()}")
print(f"  - Tem fontes variáveis no tempo: {ckt.has_time_varying()}")

print(f"\nTopologia do Buck Converter:")
print(f"  PWM(sw) --[DCR={inductor_params['DCR']*1e3:.0f}mΩ]-- mid --[L={L*1e6:.0f}µH]-- out")
print(f"                                                        |")
print(f"                                            [ESR={ESR*1e3:.0f}mΩ]--[C={C*1e6:.0f}µF]--GND")
print(f"                                                        |")
print(f"                                              [Rload={R_load:.1f}Ω]--GND")

print(f"\nParâmetros do PWM:")
print(f"  V_high = {pwm_params.v_high}V, V_low = {pwm_params.v_low}V")
print(f"  Frequência = {pwm_params.frequency/1e3:.0f}kHz")
print(f"  Duty = {pwm_params.duty:.3f} ({pwm_params.duty*100:.1f}%)")
print(f"  Período = {1/pwm_params.frequency*1e6:.1f}µs")

# Verificar forma de onda PWM
pwm_test = pulsim.PWMVoltageSource(pwm_params, "test")
T_pwm = 1/fsw
print(f"\nVerificação da forma de onda PWM (nó sw):")
print(f"  V(0) = {pwm_test.voltage_at(0):.1f}V")
print(f"  V(D*T) = {pwm_test.voltage_at(D_nom*T_pwm*0.99):.1f}V (fim do pulso ON)")
print(f"  V(D*T+) = {pwm_test.voltage_at(D_nom*T_pwm*1.01):.1f}V (início do OFF)")
print(f"  V(T) = {pwm_test.voltage_at(T_pwm*0.99):.1f}V (fim do período)")

CONSTRUÇÃO DO CIRCUITO BUCK COM PWM REAL
Circuito criado com 6 dispositivos:
  - Nós: 4 (sw, mid, out, cap)
  - Tamanho do sistema: 6
  - Tem fontes variáveis no tempo: True

Topologia do Buck Converter:
  PWM(sw) --[DCR=15mΩ]-- mid --[L=63µH]-- out
                                                        |
                                            [ESR=20mΩ]--[C=60µF]--GND
                                                        |
                                              [Rload=2.4Ω]--GND

Parâmetros do PWM:
  V_high = 48.0V, V_low = 0.0V
  Frequência = 100kHz
  Duty = 0.250 (25.0%)
  Período = 10.0µs

Verificação da forma de onda PWM (nó sw):
  V(0) = 0.0V
  V(D*T) = 48.0V (fim do pulso ON)
  V(D*T+) = 24.0V (início do OFF)
  V(T) = 0.0V (fim do período)


In [16]:
# ============================================================
# ANÁLISE DC - PONTO DE OPERAÇÃO INICIAL
# ============================================================

# Nota: Com fonte PWM, a análise DC usa o valor em t=0
# que corresponde ao estado HIGH do PWM

opts = pulsim.NewtonOptions()
opts.max_iterations = 50

x0 = np.zeros(ckt.system_size())
dc_result = pulsim.solve_dc(ckt, x0, opts)

print("Análise DC (t=0, PWM em estado HIGH):")
print(f"  Convergiu: {'✓' if dc_result.success() else '✗'}")
print(f"  Iterações: {dc_result.iterations}")

if dc_result.success():
    node_names = ckt.node_names()
    print(f"\nTensões nos nós:")
    for i, name in enumerate(node_names):
        print(f"  V({name}) = {dc_result.solution[i]:.4f} V")
    
    # Estado inicial para transiente
    x0_trans = dc_result.solution.copy()
    
    print(f"\n📌 Este é o estado inicial para a simulação transiente")
    print(f"   O capacitor começa carregado com a tensão do estado DC inicial")
else:
    x0_trans = np.zeros(ckt.system_size())
    print(f"  Erro: {dc_result.message}")
    print("  Usando estado inicial zero para transiente")

Análise DC (t=0, PWM em estado HIGH):
  Convergiu: ✓
  Iterações: 1

Tensões nos nós:
  V(sw) = 0.0000 V
  V(mid) = 0.0000 V
  V(out) = 0.0000 V
  V(cap) = 0.0000 V

📌 Este é o estado inicial para a simulação transiente
   O capacitor começa carregado com a tensão do estado DC inicial


In [17]:
# ============================================================
# SIMULAÇÃO TRANSIENTE COM PWM REAL
# ============================================================

# Calcular tempo necessário para estabilização do filtro LC
# f0 = 1/(2π√LC), Q = R×√(C/L), τ_settle ≈ Q/f0
f0_lc = 1/(2*np.pi*np.sqrt(L*C))
Q_lc = R_load * np.sqrt(C/L)
tau_settle = Q_lc / f0_lc

print("="*60)
print("SIMULAÇÃO TRANSIENTE COM PWM")
print("="*60)
print(f"\nAnálise do filtro LC:")
print(f"  Frequência de ressonância: f0 = {f0_lc:.0f} Hz")
print(f"  Fator de qualidade: Q = {Q_lc:.2f}")
print(f"  Tempo de estabelecimento (τ): {tau_settle*1e3:.2f} ms")

# Simular 5τ para garantir regime permanente
T_pwm = 1/fsw
t_sim = max(5 * tau_settle, 50 * T_pwm)  # Pelo menos 5τ ou 50 períodos
n_periods = int(t_sim / T_pwm)

# Timestep: 20-50 pontos por período PWM
points_per_period = 30
dt = T_pwm / points_per_period

print(f"\nParâmetros de simulação:")
print(f"  Período PWM: T = {T_pwm*1e6:.2f}µs")
print(f"  Tempo de simulação: {t_sim*1e3:.2f}ms ({n_periods} períodos)")
print(f"  Timestep: dt = {dt*1e9:.0f}ns ({points_per_period} pontos/período)")
print(f"  Total de pontos: ~{int(t_sim/dt)}")

# Condições iniciais: começar do zero para ver partida
x0_trans = np.zeros(ckt.system_size())

# Configurar solver (kernel novo)
newton_opts = pulsim.NewtonOptions()
newton_opts.max_iterations = 50
newton_opts.num_nodes = ckt.num_nodes()
newton_opts.num_branches = ckt.num_branches()

linear_solver = pulsim.LinearSolverStackConfig.defaults()
linear_solver.order = [pulsim.LinearSolverKind.KLU, pulsim.LinearSolverKind.GMRES]
linear_solver.allow_fallback = True
linear_solver.auto_select = True

# Executar simulação transiente
print(f"\nSimulando...")
times, states, success, msg = pulsim.run_transient(
    ckt, 0.0, t_sim, dt, x0_trans, newton_opts, linear_solver
)

print(f"Resultado: {'✓ Sucesso' if success else '✗ Falha - ' + msg}")
print(f"Pontos simulados: {len(times)}")

if success:
    states_arr = np.array(states)
    node_names = ckt.node_names()
    
    print(f"\nNós disponíveis: {node_names}")
    
    # Extrair tensões dos nós
    idx_sw = node_names.index('sw') if 'sw' in node_names else -1
    idx_out = node_names.index('out') if 'out' in node_names else -1
    
    V_sw = states_arr[:, idx_sw] if idx_sw >= 0 else None
    V_out = states_arr[:, idx_out] if idx_out >= 0 else states_arr[:, 0]
    
    # Análise em regime permanente (últimos 20%)
    idx_steady = int(0.8 * len(times))
    V_out_steady = V_out[idx_steady:]
    
    print(f"\nV_sw (nó de chaveamento - DEVE comutar):")
    if V_sw is not None:
        print(f"  Min: {np.min(V_sw):.1f}V, Max: {np.max(V_sw):.1f}V")
    
    print(f"\nV_out (saída filtrada - NÃO deve comutar):")
    print(f"  Inicial: {V_out[0]:.3f}V")
    print(f"  Final: {V_out[-1]:.3f}V")
    print(f"  Média (regime): {np.mean(V_out_steady):.3f}V")
    print(f"  Min/Max (regime): {np.min(V_out_steady):.3f}V / {np.max(V_out_steady):.3f}V")
    print(f"  Ripple: {(np.max(V_out_steady) - np.min(V_out_steady))*1e3:.1f}mV")
    print(f"\n  V_out esperado (D×Vin): {D_nom * Vin_nom:.2f}V")
    print(f"  Erro: {abs(np.mean(V_out_steady) - D_nom*Vin_nom)/(D_nom*Vin_nom)*100:.1f}%")

SIMULAÇÃO TRANSIENTE COM PWM

Análise do filtro LC:
  Frequência de ressonância: f0 = 2589 Hz
  Fator de qualidade: Q = 2.34
  Tempo de estabelecimento (τ): 0.90 ms

Parâmetros de simulação:
  Período PWM: T = 10.00µs
  Tempo de simulação: 4.52ms (452 períodos)
  Timestep: dt = 333ns (30 pontos/período)
  Total de pontos: ~13571

Simulando...
Resultado: ✓ Sucesso
Pontos simulados: 13573

Nós disponíveis: ['sw', 'mid', 'out', 'cap']

V_sw (nó de chaveamento - DEVE comutar):
  Min: 0.0V, Max: 48.0V

V_out (saída filtrada - NÃO deve comutar):
  Inicial: 0.000V
  Final: 11.142V
  Média (regime): 11.130V
  Min/Max (regime): 11.107V / 11.144V
  Ripple: 36.9mV

  V_out esperado (D×Vin): 12.00V
  Erro: 7.2%


In [18]:
# ============================================================
# VISUALIZAÇÃO - FORMAS DE ONDA PWM
# ============================================================

if success:
    t_us = np.array(times) * 1e6  # Converter para µs
    t_ms = np.array(times) * 1e3  # Converter para ms
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # 1. Tensão no nó de chaveamento (PWM) - deve comutar
    ax = axes[0]
    if V_sw is not None:
        ax.plot(t_us, V_sw, 'b-', linewidth=0.5, label='V_sw (nó de chaveamento)')
    ax.axhline(Vin_nom, color='r', linestyle='--', alpha=0.5, label=f'Vin = {Vin_nom}V')
    ax.axhline(0, color='gray', linestyle='-', alpha=0.3)
    ax.set_xlabel('Tempo [µs]')
    ax.set_ylabel('Tensão [V]')
    ax.set_title('Tensão no Nó de Chaveamento (V_sw) - DEVE Comutar')
    ax.legend(loc='right')
    ax.set_ylim([-5, Vin_nom + 10])
    ax.set_xlim([0, min(t_us[-1], 50)])  # Mostrar primeiros 50µs (5 períodos)
    
    # 2. Tensão de saída - NÃO deve comutar (filtrada pelo LC)
    ax = axes[1]
    ax.plot(t_ms, V_out, 'g-', linewidth=1.5, label='V_out (saída filtrada)')
    ax.axhline(Vout, color='r', linestyle='--', alpha=0.7, label=f'V_out alvo = {Vout}V')
    ax.axhline(D_nom * Vin_nom, color='orange', linestyle=':', alpha=0.7, 
               label=f'D×Vin = {D_nom * Vin_nom:.1f}V')
    ax.set_xlabel('Tempo [ms]')
    ax.set_ylabel('Tensão [V]')
    ax.set_title('Tensão de Saída (V_out) - Filtrada pelo LC (NÃO deve comutar)')
    ax.legend(loc='right')
    
    # 3. Zoom nos últimos períodos para ver ripple
    ax = axes[2]
    n_zoom = 5  # últimos 5 períodos
    idx_start = -n_zoom * points_per_period
    t_zoom = (np.array(times[idx_start:]) - times[idx_start]) * 1e6  # Reset para 0, em µs
    v_zoom = V_out[idx_start:]
    
    ax.plot(t_zoom, v_zoom, 'g-', linewidth=1.5, label='V_out')
    ax.axhline(np.mean(v_zoom), color='r', linestyle='--', alpha=0.7, 
               label=f'Média = {np.mean(v_zoom):.3f}V')
    
    # Marcar ripple
    v_max = np.max(v_zoom)
    v_min = np.min(v_zoom)
    ripple = v_max - v_min
    ax.fill_between(t_zoom, v_min, v_max, alpha=0.2, color='purple', label=f'Ripple = {ripple*1e3:.1f}mV')
    
    ax.set_xlabel('Tempo [µs]')
    ax.set_ylabel('Tensão [V]')
    ax.set_title(f'Zoom - Últimos {n_zoom} Períodos PWM (Ripple de Tensão)')
    ax.legend(loc='right')
    
    plt.tight_layout()
    plt.show()
    
    # Métricas
    print(f"\nMétricas de regime permanente (últimos {n_zoom} períodos):")
    print(f"  V_out médio: {np.mean(v_zoom):.4f}V (esperado: {Vout:.2f}V)")
    print(f"  V_out max: {v_max:.4f}V")
    print(f"  V_out min: {v_min:.4f}V")
    print(f"  Ripple: {ripple*1e3:.2f}mV (especificado: <{delta_Vout_max*1e3:.0f}mV)")
    print(f"  Erro da média: {abs(np.mean(v_zoom) - Vout)/Vout*100:.2f}%")


Métricas de regime permanente (últimos 5 períodos):
  V_out médio: 11.1304V (esperado: 12.00V)
  V_out max: 11.1441V
  V_out min: 11.1072V
  Ripple: 36.88mV (especificado: <50mV)
  Erro da média: 7.25%


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_88225/1705323169.py:58: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 5.2 Comparação Teoria vs Simulação

In [19]:
# ============================================================
# COMPARAÇÃO TEORIA vs SIMULAÇÃO PWM
# ============================================================

print("="*60)
print("COMPARAÇÃO: TEORIA vs SIMULAÇÃO PWM")
print("="*60)

if success and len(times) > 10:
    # Valores em regime permanente (últimos 20%)
    idx_steady = int(0.8 * len(times))
    V_out_steady = V_out[idx_steady:]
    
    Vout_sim_avg = np.mean(V_out_steady)
    Vout_sim_max = np.max(V_out_steady)
    Vout_sim_min = np.min(V_out_steady)
    ripple_sim = Vout_sim_max - Vout_sim_min
    
    print(f"\nAnálise em regime permanente (últimos {100-80}% da simulação):")
    print(f"\n{'Parâmetro':<25} {'Teórico':>12} {'Simulado':>12} {'Erro':>10}")
    print("-" * 61)
    print(f"{'Vout médio [V]':<25} {Vout:>12.3f} {Vout_sim_avg:>12.3f} {abs(Vout-Vout_sim_avg)/Vout*100:>9.2f}%")
    print(f"{'Ripple Vout [mV]':<25} {delta_Vout_real*1e3:>12.1f} {ripple_sim*1e3:>12.1f} {'N/A':>10}")
    
    # Duty cycle efetivo (calculado a partir da tensão média)
    D_effective = Vout_sim_avg / Vin_nom
    print(f"{'Duty efetivo':<25} {D_nom:>12.3f} {D_effective:>12.3f} {abs(D_nom-D_effective)/D_nom*100:>9.2f}%")
    
    # Corrente de saída
    I_out_sim = Vout_sim_avg / R_load
    print(f"{'Iout [A]':<25} {Iout_nom:>12.2f} {I_out_sim:>12.2f} {abs(Iout_nom-I_out_sim)/Iout_nom*100:>9.2f}%")
    
    print(f"\n✅ A simulação PWM mostra:")
    print(f"   - V_sw comutando entre 0V e {Vin_nom}V (correto)")
    print(f"   - V_out filtrado pelo LC (tensão DC com ripple)")
    print(f"   - Valor médio ≈ D × Vin = {D_nom:.3f} × {Vin_nom}V = {D_nom*Vin_nom:.1f}V")
    
    # Armazenar para uso posterior
    Vout_sim = Vout_sim_avg
    delta_Vout_sim = ripple_sim
    IL_sim_avg = I_out_sim
else:
    print("Simulação não disponível")
    Vout_sim = Vout
    delta_Vout_sim = delta_Vout_real
    IL_sim_avg = Iout_nom

COMPARAÇÃO: TEORIA vs SIMULAÇÃO PWM

Análise em regime permanente (últimos 20% da simulação):

Parâmetro                      Teórico     Simulado       Erro
-------------------------------------------------------------
Vout médio [V]                  12.000       11.130      7.25%
Ripple Vout [mV]                  60.5         36.9        N/A
Duty efetivo                     0.250        0.232      7.25%
Iout [A]                          5.00         4.64      7.25%

✅ A simulação PWM mostra:
   - V_sw comutando entre 0V e 48.0V (correto)
   - V_out filtrado pelo LC (tensão DC com ripple)
   - Valor médio ≈ D × Vin = 0.250 × 48.0V = 12.0V


In [20]:
# ============================================================
# DEMONSTRAÇÃO: CONTROLE PI EM MALHA FECHADA
# ============================================================

print("="*60)
print("SIMULAÇÃO COM CONTROLADOR PI EM MALHA FECHADA")
print("="*60)

# Criar novo circuito para controle em malha fechada
# Mesma topologia correta: sw -> DCR -> L -> out -> ESR -> C -> gnd
ckt_cl = pulsim.Circuit()

n_sw_cl = ckt_cl.add_node("sw")
n_mid_cl = ckt_cl.add_node("mid")
n_out_cl = ckt_cl.add_node("out")
n_cap_cl = ckt_cl.add_node("cap")

# PWM com duty inicial baixo (vai ser ajustado pelo PI)
pwm_cl_params = pulsim.PWMParams()
pwm_cl_params.v_high = Vin_nom
pwm_cl_params.v_low = 0.0
pwm_cl_params.frequency = fsw
pwm_cl_params.duty = 0.15  # Começar com duty menor (vai subir para ~0.25)

ckt_cl.add_pwm_voltage_source("Vpwm", n_sw_cl, ckt_cl.ground(), pwm_cl_params)

# Topologia correta: sw -> DCR -> L -> out
ckt_cl.add_resistor("Rdcr", n_sw_cl, n_mid_cl, inductor_params['DCR'])
ckt_cl.add_inductor("L1", n_mid_cl, n_out_cl, L)

# Capacitor com ESR: out -> ESR -> C -> gnd
ckt_cl.add_resistor("Resr", n_out_cl, n_cap_cl, ESR)
ckt_cl.add_capacitor("Cout", n_cap_cl, ckt_cl.ground(), C)

# Carga
ckt_cl.add_resistor("Rload", n_out_cl, ckt_cl.ground(), R_load)

# Controlador PI para malha fechada
# Usar ganhos mais conservadores para simulação PWM
pi_controller = pulsim.PIController(
    Kp=0.0142,            # Ganho proporcional (reprojetado: fc=318Hz)
    Ki=28.39,            # Ganho integral (reprojetado: PM=128°)
    output_min=0.10,    # Duty mínimo 10%
    output_max=0.50     # Duty máximo 50%
)

V_ref = Vout  # Referência de tensão = 12V

print(f"Controlador PI configurado:")
print(f"  Kp = {pi_controller.Kp():.4f}")
print(f"  Ki = {pi_controller.Ki():.4f}")
print(f"  Limites de duty: [{pi_controller.output_min():.2f}, {pi_controller.output_max():.2f}]")
print(f"  Referência: V_ref = {V_ref}V")

# Configurar solver (kernel novo)
newton_opts_cl = pulsim.NewtonOptions()
newton_opts_cl.max_iterations = 50
newton_opts_cl.num_nodes = ckt_cl.num_nodes()
newton_opts_cl.num_branches = ckt_cl.num_branches()

linear_solver_cl = pulsim.LinearSolverStackConfig.defaults()
linear_solver_cl.order = [pulsim.LinearSolverKind.KLU, pulsim.LinearSolverKind.GMRES]
linear_solver_cl.allow_fallback = True
linear_solver_cl.auto_select = True

# Histórico para plot
time_hist = []
vout_hist = []
duty_hist = []
error_hist = []

# Simulação manual com update de PI a cada período PWM
T_pwm = 1/fsw
n_periods_cl = 1000  # Simular 1000 períodos (10ms) para convergência
dt_cl = T_pwm / 50  # 50 pontos por período

x = np.zeros(ckt_cl.system_size())
t = 0.0

print(f"\nSimulando {n_periods_cl} períodos PWM com controle PI...")
print(f"Tempo total: {n_periods_cl * T_pwm * 1e3:.1f}ms")

node_names_cl = ckt_cl.node_names()
idx_out_cl = node_names_cl.index('out')

for period in range(n_periods_cl):
    # Simular um período PWM
    times_p, states_p, ok, err_msg = pulsim.run_transient(
        ckt_cl, t, t + T_pwm, dt_cl, x, newton_opts_cl, linear_solver_cl
    )
    
    if not ok:
        print(f"Falha no período {period}: {err_msg}")
        break
    
    # Atualizar estado
    states_arr_p = np.array(states_p)
    x = states_arr_p[-1]
    
    # Medir tensão de saída (média do período)
    V_out_period = np.mean(states_arr_p[:, idx_out_cl])
    
    # Atualizar controlador PI
    t += T_pwm
    error = V_ref - V_out_period
    new_duty = pi_controller.update(error, t)
    
    # Aplicar novo duty ao PWM
    ckt_cl.set_pwm_duty("Vpwm", new_duty)
    
    # Registrar
    time_hist.append(t * 1e3)  # ms
    vout_hist.append(V_out_period)
    duty_hist.append(new_duty)
    error_hist.append(error)
    
    # Progress
    if (period + 1) % 50 == 0:
        print(f"  Período {period+1}: V_out={V_out_period:.2f}V, D={new_duty:.3f}")

print(f"\nSimulação concluída!")
print(f"  V_out final: {vout_hist[-1]:.3f}V (ref: {V_ref}V)")
print(f"  Duty final: {duty_hist[-1]:.3f} ({duty_hist[-1]*100:.1f}%)")
print(f"  Erro final: {error_hist[-1]*1e3:.1f}mV")

SIMULAÇÃO COM CONTROLADOR PI EM MALHA FECHADA
Controlador PI configurado:
  Kp = 0.0142
  Ki = 28.3900
  Limites de duty: [0.10, 0.50]
  Referência: V_ref = 12.0V

Simulando 1000 períodos PWM com controle PI...
Tempo total: 10.0ms
  Período 50: V_out=8.20V, D=0.136
  Período 100: V_out=8.62V, D=0.187


  Período 150: V_out=9.55V, D=0.213


  Período 200: V_out=10.90V, D=0.220
  Período 250: V_out=11.06V, D=0.231
  Período 300: V_out=11.33V, D=0.239
  Período 350: V_out=11.69V, D=0.241
  Período 400: V_out=11.48V, D=0.247


  Período 450: V_out=11.54V, D=0.253
  Período 500: V_out=11.57V, D=0.259
  Período 550: V_out=11.78V, D=0.260
  Período 600: V_out=11.83V, D=0.261


  Período 650: V_out=12.07V, D=0.258
  Período 700: V_out=11.79V, D=0.262
  Período 750: V_out=12.18V, D=0.257
  Período 800: V_out=11.79V, D=0.263


  Período 850: V_out=12.19V, D=0.257
  Período 900: V_out=11.79V, D=0.263


  Período 950: V_out=12.19V, D=0.257
  Período 1000: V_out=11.79V, D=0.263

Simulação concluída!
  V_out final: 11.792V (ref: 12.0V)
  Duty final: 0.263 (26.3%)
  Erro final: 208.3mV


In [21]:
# ============================================================
# VISUALIZAÇÃO DO CONTROLE EM MALHA FECHADA
# ============================================================

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# 1. Tensão de saída
ax = axes[0]
ax.plot(time_hist, vout_hist, 'b-', linewidth=1.5, label='V_out')
ax.axhline(V_ref, color='r', linestyle='--', alpha=0.7, label=f'V_ref = {V_ref}V')
ax.axhline(V_ref * 1.02, color='g', linestyle=':', alpha=0.5, label='±2% banda')
ax.axhline(V_ref * 0.98, color='g', linestyle=':', alpha=0.5)
ax.set_ylabel('Tensão [V]')
ax.set_title('Resposta do Controlador PI em Malha Fechada')
ax.legend(loc='right')
ax.set_ylim([0, V_ref * 1.5])

# 2. Duty cycle
ax = axes[1]
ax.plot(time_hist, [d*100 for d in duty_hist], 'orange', linewidth=1.5, label='Duty')
ax.axhline(D_nom*100, color='gray', linestyle='--', alpha=0.5, 
           label=f'D nominal = {D_nom*100:.1f}%')
ax.set_ylabel('Duty Cycle [%]')
ax.legend(loc='right')
ax.set_ylim([0, 100])

# 3. Erro
ax = axes[2]
ax.plot(time_hist, [e*1e3 for e in error_hist], 'm-', linewidth=1.5, label='Erro')
ax.axhline(0, color='gray', linestyle='-', alpha=0.3)
ax.axhline(50, color='g', linestyle=':', alpha=0.5, label='±50mV')
ax.axhline(-50, color='g', linestyle=':', alpha=0.5)
ax.set_xlabel('Tempo [ms]')
ax.set_ylabel('Erro [mV]')
ax.legend(loc='right')

plt.tight_layout()
plt.show()

# Métricas de desempenho
# Tempo de estabelecimento (2% banda)
v_settled_band = V_ref * 0.02
try:
    idx_settled = None
    for i in range(len(vout_hist)-1, -1, -1):
        if abs(vout_hist[i] - V_ref) > v_settled_band:
            idx_settled = i + 1
            break
    t_settling = time_hist[idx_settled] if idx_settled else time_hist[0]
except:
    t_settling = time_hist[-1]

# Overshoot
v_max_cl = max(vout_hist)
overshoot = (v_max_cl - V_ref) / V_ref * 100 if v_max_cl > V_ref else 0

print(f"\nMétricas de desempenho do controlador:")
print(f"  Tempo de estabelecimento (2%): {t_settling:.2f}ms")
print(f"  Overshoot: {overshoot:.1f}%")
print(f"  Erro em regime permanente: {error_hist[-1]*1e3:.2f}mV")
print(f"  Duty em regime permanente: {duty_hist[-1]*100:.2f}%")


Métricas de desempenho do controlador:
  Tempo de estabelecimento (2%): 6.45ms
  Overshoot: 1.8%
  Erro em regime permanente: 208.26mV
  Duty em regime permanente: 26.27%


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_88225/2682645523.py:38: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 5.3 Análise de Perdas via Simulação

In [22]:
# ============================================================
# CÁLCULO DE PERDAS COM PULSIM LOSS API
# ============================================================

print("="*60)
print("ANÁLISE DE PERDAS COM PULSIM")
print("="*60)

# Criar parâmetros de perda do MOSFET
mosfet_loss = pulsim.MOSFETLossParams()
mosfet_loss.Rds_on = mosfet_params['Rds_on']
mosfet_loss.Rds_on_tc = 0.006  # Coeficiente de temperatura típico
mosfet_loss.T_ref = 25.0
mosfet_loss.Eon_25C = 50e-6    # Energia de turn-on @ 25°C (estimativa)
mosfet_loss.Eoff_25C = 30e-6   # Energia de turn-off @ 25°C (estimativa)
mosfet_loss.Esw_tc = 0.003     # TC das perdas de chaveamento
mosfet_loss.V_ref = Vin_nom
mosfet_loss.I_ref = IL_avg
mosfet_loss.Qg = mosfet_params['Qg']

print("Parâmetros MOSFET configurados:")
print(f"  Rds_on = {mosfet_loss.Rds_on*1e3:.0f}mΩ @ 25°C")
print(f"  Rds_on @ 100°C = {mosfet_loss.Rds_on_at_T(100)*1e3:.1f}mΩ")

# Calcular perdas de condução usando a API
T_junction = 100  # °C (temperatura de operação estimada)
I_rms = IL_avg * np.sqrt(D_nom)  # Corrente RMS no MOSFET

P_cond_pulsim = pulsim.ConductionLoss.mosfet(I_rms, mosfet_loss, T_junction)
print(f"\nPerda de condução (Pulsim): {P_cond_pulsim:.3f}W")
print(f"Perda de condução (teórico): {P_cond_mosfet:.3f}W")

# Calcular perdas de chaveamento
# Signature: mosfet_power(I, V, T, f_sw, params)
P_sw_pulsim = pulsim.SwitchingLoss.mosfet_power(
    IL_avg,      # Corrente de carga
    Vin_nom,     # Tensão de bloqueio
    T_junction,  # Temperatura
    fsw,         # Frequência de chaveamento  
    mosfet_loss  # Parâmetros
)
print(f"\nPerda de chaveamento (Pulsim): {P_sw_pulsim:.3f}W")
print(f"Perda de chaveamento (teórico): {P_sw_mosfet:.3f}W")

# Total MOSFET
P_mosfet_pulsim = P_cond_pulsim + P_sw_pulsim
print(f"\nPerda total MOSFET:")
print(f"  Pulsim: {P_mosfet_pulsim:.3f}W")
print(f"  Teórico: {P_mosfet_total:.3f}W")

# Criar parâmetros de perda do diodo
diode_loss = pulsim.DiodeLossParams()
diode_loss.Vf = diode_params['Vf']  # Tensão direta @ 25°C
diode_loss.Rd = 0.02  # Resistência dinâmica típica
diode_loss.Qrr = diode_params['Qrr']
diode_loss.trr = diode_params['trr']

# Perda de condução do diodo
# Signature: diode(I, params, T) -> P = Vf * I + Rd * I²
I_diode_avg_calc = IL_avg * (1 - D_nom)
P_diode_cond_pulsim = pulsim.ConductionLoss.diode(
    I_diode_avg_calc,  # Corrente média
    diode_loss, 
    T_junction
)
print(f"\nPerda condução diodo (Pulsim): {P_diode_cond_pulsim:.3f}W")
print(f"Perda condução diodo (teórico): {P_cond_diode:.3f}W")

# Perda de recuperação reversa
# Signature: diode_power(If, Vr, T, f_sw, params)
P_diode_rr_pulsim = pulsim.SwitchingLoss.diode_power(
    I_diode_avg_calc,  # Corrente direta
    Vin_nom,           # Tensão reversa
    T_junction,        # Temperatura
    fsw,               # Frequência
    diode_loss         # Parâmetros
)
print(f"Perda recuperação (Pulsim): {P_diode_rr_pulsim:.3f}W")
print(f"Perda recuperação (teórico): {P_rr_diode:.3f}W")

# Resumo de eficiência
P_total_pulsim = P_mosfet_pulsim + P_diode_cond_pulsim + P_diode_rr_pulsim
P_out_calc = Vout * Iout_nom
eta_pulsim = P_out_calc / (P_out_calc + P_total_pulsim)

print(f"\n{'='*40}")
print(f"RESUMO (Pulsim vs Teórico)")
print(f"{'='*40}")
print(f"  Perdas MOSFET:  {P_mosfet_pulsim:.2f}W vs {P_mosfet_total:.2f}W")
print(f"  Perdas Diodo:   {P_diode_cond_pulsim + P_diode_rr_pulsim:.2f}W vs {P_diode_total:.2f}W")
print(f"  Eficiência:     {eta_pulsim*100:.1f}% vs {eta*100:.1f}%")

ANÁLISE DE PERDAS COM PULSIM
Parâmetros MOSFET configurados:
  Rds_on = 44mΩ @ 25°C
  Rds_on @ 100°C = 63.8mΩ

Perda de condução (Pulsim): 0.399W
Perda de condução (teórico): 0.481W

Perda de chaveamento (Pulsim): 9.800W
Perda de chaveamento (teórico): 1.116W

Perda total MOSFET:
  Pulsim: 10.199W
  Teórico: 1.710W

Perda condução diodo (Pulsim): 3.094W
Perda condução diodo (teórico): 2.812W
Perda recuperação (Pulsim): 0.240W
Perda recuperação (teórico): 0.240W

RESUMO (Pulsim vs Teórico)
  Perdas MOSFET:  10.20W vs 1.71W
  Perdas Diodo:   3.33W vs 3.05W
  Eficiência:     81.6% vs 92.1%


### 5.4 Simulação Térmica

In [23]:
# ============================================================
# SIMULAÇÃO TÉRMICA COM PULSIM
# ============================================================

print("="*60)
print("SIMULAÇÃO TÉRMICA COM PULSIM")
print("="*60)

# Criar modelo térmico Foster para o MOSFET
# Rede térmica: Junção -> Case -> Sink -> Ambiente
mosfet_thermal = pulsim.create_mosfet_thermal_model(
    Rth_jc=mosfet_params['Rth_jc'],  # °C/W
    Rth_cs=mosfet_params['Rth_cs'],  # °C/W
    Rth_sa=5.0,                       # °C/W (dissipador)
    name="MOSFET"
)

print(f"Modelo térmico MOSFET criado:")
print(f"  Rth total: {mosfet_thermal.total_Rth():.2f} °C/W")
print(f"  Estágios: {mosfet_thermal.num_stages()}")

# Criar modelo térmico para o diodo
diode_thermal = pulsim.create_mosfet_thermal_model(
    Rth_jc=diode_params['Rth_jc'],
    Rth_cs=0.5,
    Rth_sa=5.0,
    name="Diodo"
)

print(f"\nModelo térmico Diodo criado:")
print(f"  Rth total: {diode_thermal.total_Rth():.2f} °C/W")

# Criar simuladores térmicos
sim_mosfet = pulsim.ThermalSimulator(mosfet_thermal)
sim_diode = pulsim.ThermalSimulator(diode_thermal)

# Configurar temperatura ambiente
sim_mosfet.set_ambient(Ta)
sim_diode.set_ambient(Ta)

# Simular aquecimento por 60 segundos
print(f"\nSimulando aquecimento térmico...")
print(f"  Temperatura ambiente: {Ta}°C")
print(f"  Potência MOSFET: {P_mosfet_total:.2f}W")
print(f"  Potência Diodo: {P_diode_total:.2f}W")

t_total = 60.0  # segundos
dt = 0.1        # passo de 100ms

time_thermal = []
Tj_mosfet_hist = []
Tj_diode_hist = []

t = 0.0
while t < t_total:
    # Step térmico
    sim_mosfet.step(P_mosfet_total, dt)
    sim_diode.step(P_diode_total, dt)
    
    # Registrar (Tj é método, não propriedade)
    time_thermal.append(t)
    Tj_mosfet_hist.append(sim_mosfet.Tj())
    Tj_diode_hist.append(sim_diode.Tj())
    
    t += dt

# Valores finais (regime permanente)
Tj_mosfet_final = Tj_mosfet_hist[-1]
Tj_diode_final = Tj_diode_hist[-1]

# Temperatura de regime calculada analiticamente
Tj_mosfet_ss = sim_mosfet.steady_state_temperature(P_mosfet_total)
Tj_diode_ss = sim_diode.steady_state_temperature(P_diode_total)

print(f"\nTemperaturas em regime permanente:")
print(f"  Tj MOSFET: {Tj_mosfet_final:.1f}°C (analítico: {Tj_mosfet_ss:.1f}°C)")
print(f"  Tj Diodo: {Tj_diode_final:.1f}°C (analítico: {Tj_diode_ss:.1f}°C)")

# Verificar limites
Tj_max_limit = 150  # °C
if Tj_mosfet_final > Tj_max_limit:
    print(f"  ⚠️ MOSFET excede Tj_max = {Tj_max_limit}°C!")
else:
    print(f"  ✓ MOSFET dentro do limite ({Tj_max_limit - Tj_mosfet_final:.0f}°C de margem)")

if Tj_diode_final > Tj_max_limit:
    print(f"  ⚠️ Diodo excede Tj_max = {Tj_max_limit}°C!")
else:
    print(f"  ✓ Diodo dentro do limite ({Tj_max_limit - Tj_diode_final:.0f}°C de margem)")

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(time_thermal, Tj_mosfet_hist, 'r-', linewidth=2, 
        label=f'MOSFET (P={P_mosfet_total:.2f}W)')
ax.plot(time_thermal, Tj_diode_hist, 'b-', linewidth=2, 
        label=f'Diodo (P={P_diode_total:.2f}W)')
ax.axhline(Ta, color='gray', linestyle='--', alpha=0.5, 
           label=f'T ambiente = {Ta}°C')
ax.axhline(Tj_max_limit, color='red', linestyle=':', alpha=0.7, 
           label=f'Tj max = {Tj_max_limit}°C')

ax.set_xlabel('Tempo [s]')
ax.set_ylabel('Temperatura de Junção [°C]')
ax.set_title('Resposta Térmica Transitória - Simulação Pulsim')
ax.legend(loc='lower right')
ax.set_xlim([0, t_total])
ax.set_ylim([Ta - 5, max(Tj_mosfet_final, Tj_diode_final) + 20])
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

SIMULAÇÃO TÉRMICA COM PULSIM
Modelo térmico MOSFET criado:
  Rth total: 6.50 °C/W
  Estágios: 3

Modelo térmico Diodo criado:
  Rth total: 7.50 °C/W

Simulando aquecimento térmico...
  Temperatura ambiente: 40.0°C
  Potência MOSFET: 1.71W
  Potência Diodo: 3.05W

Temperaturas em regime permanente:
  Tj MOSFET: 51.1°C (analítico: 51.1°C)
  Tj Diodo: 62.9°C (analítico: 62.9°C)
  ✓ MOSFET dentro do limite (99°C de margem)
  ✓ Diodo dentro do limite (87°C de margem)


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_88225/2273485734.py:112: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 6. Conclusões

In [24]:
# ============================================================
# RESUMO FINAL DO PROJETO
# ============================================================

print("="*70)
print("RESUMO FINAL DO PROJETO - CONVERSOR BUCK COM PWM")
print("="*70)

print("\n1. ESPECIFICAÇÕES ATENDIDAS:")
print(f"   {'Parâmetro':<30} {'Especificado':>15} {'Simulado':>15}")
print("   " + "-"*62)
print(f"   {'Tensão de saída':<30} {Vout:>12.1f}V {vout_hist[-1]:>12.2f}V")
print(f"   {'Ripple de tensão':<30} {'<'+str(int(delta_Vout_max*1e3))+'mV':>15} {ripple_sim*1e3:>12.1f}mV")
print(f"   {'Eficiência (estimada)':<30} {'>' + str(int(eta_target*100)) + '%':>15} {eta*100:>12.1f}%")

print("\n2. COMPONENTES SELECIONADOS:")
print(f"   Indutor: L = {L*1e6:.0f}µH (DCR = {inductor_params['DCR']*1e3:.0f}mΩ)")
print(f"   Capacitor: C = {C*1e6:.0f}µF (ESR = {ESR*1e3:.0f}mΩ)")
print(f"   MOSFET: {mosfet_params['name']} (Rds_on = {mosfet_params['Rds_on']*1e3:.0f}mΩ)")
print(f"   Diodo: {diode_params['name']} (Vf = {diode_params['Vf']}V)")

print("\n3. CONTROLADOR PI:")
print(f"   Kp = {Kp:.4f}")
print(f"   Ki = {Ki:.4f}")
print(f"   Margem de fase (projeto): {PM_real:.1f}°")
print(f"   Frequência de cruzamento: {fc_real/1e3:.1f}kHz")

print("\n4. DESEMPENHO DO CONTROLE (Simulação PWM):")
print(f"   Tempo de estabelecimento: {t_settling:.2f}ms")
print(f"   Overshoot: {overshoot:.1f}%")
print(f"   Erro em regime: {error_hist[-1]*1e3:.1f}mV")
print(f"   Duty final: {duty_hist[-1]*100:.1f}%")

print("\n5. PERDAS (Análise Teórica):")
print(f"   MOSFET: {P_mosfet_total:.2f}W")
print(f"   Diodo: {P_diode_total:.2f}W")
print(f"   Indutor: {P_inductor_total:.2f}W")
print(f"   Total: {P_total:.2f}W")

print("\n6. TÉRMICO:")
print(f"   Tj MOSFET (estimado): {Tj_mosfet_final:.1f}°C")
print(f"   Tj Diodo (estimado): {Tj_diode_final:.1f}°C")
print(f"   Dissipador: Rth_sa = 5.0°C/W")

print("\n" + "="*70)
print("PROJETO CONCLUÍDO COM SIMULAÇÃO PWM REAL!")
print("="*70)
print("\n✅ Novos recursos utilizados:")
print("   - PWMVoltageSource: fonte de tensão com chaveamento real")
print("   - PIController: controlador PI com anti-windup")
print("   - Simulação transiente com atualização de duty em tempo real")

RESUMO FINAL DO PROJETO - CONVERSOR BUCK COM PWM

1. ESPECIFICAÇÕES ATENDIDAS:
   Parâmetro                         Especificado        Simulado
   --------------------------------------------------------------
   Tensão de saída                        12.0V        11.79V
   Ripple de tensão                         <50mV         36.9mV
   Eficiência (estimada)                     >95%         92.1%

2. COMPONENTES SELECIONADOS:
   Indutor: L = 63µH (DCR = 15mΩ)
   Capacitor: C = 60µF (ESR = 20mΩ)
   MOSFET: IRF540N (Rds_on = 44mΩ)
   Diodo: MUR860 (Vf = 0.9V)

3. CONTROLADOR PI:
   Kp = 0.2889
   Ki = 2349.0823
   Margem de fase (projeto): 3.7°
   Frequência de cruzamento: 10.0kHz

4. DESEMPENHO DO CONTROLE (Simulação PWM):
   Tempo de estabelecimento: 6.45ms
   Overshoot: 1.8%
   Erro em regime: 208.3mV
   Duty final: 26.3%

5. PERDAS (Análise Teórica):
   MOSFET: 1.71W
   Diodo: 3.05W
   Indutor: 0.42W
   Total: 5.18W

6. TÉRMICO:
   Tj MOSFET (estimado): 51.1°C
   Tj Diodo (estimado